In [1]:
"""This is video reader with all operations inside fnctions, so they can be called on 
    each frame out of a set of frames, and each video out of a set of videos
    """

'This is video reader with all operations inside fnctions, so they can be called on \n    each frame out of a set of frames, and each video out of a set of videos\n    '

In [2]:
"""Imports"""
import matplotlib.pyplot as plt #for maknig plots inside the notebook
import skimage
import skvideo.io
from skvideo.io import vreader, ffprobe
from skimage import measure, morphology, feature
from skimage.filters import *
from skimage.morphology import *
from operator import attrgetter
import numpy as np
import math
from itertools import chain
#from functools import partial, update_wrapper
from scipy import ndimage 
from glob import glob
import seaborn as sns
from collections import OrderedDict 
#import antigravity
blob_doh = skimage.feature.blob_doh
blob_dog = skimage.feature.blob_dog

In [3]:
"""Functions"""
# apply total thresholding to each of a list of frames; filter slides with wrong number

def total_threshold_filter(frame, frame_no, transition_threshold, class_thresh,
                           broken_count, class_num, origin, last_class, last_whole, state_start, filtrate_len, 
                           classes, transitions, times): 
    
   
    
   # ---setup image and detect blobs ----------------
    thresh_img = frame > threshold_isodata(frame)# binary image
        
    phighlight = binary_opening(thresh_img, disk(7))
    img = np.copy(frame)
    img[phighlight==0] = 0
    pblobs = blob_dog(img, min_sigma = 8, max_sigma = 12, threshold = 0.05, overlap=0.3)

    ghighlight = binary_opening(
    morphology.white_tophat(gaussian(thresh_img, sigma=0.3), disk(7)),disk(3))
    img = np.copy(frame)
    img[ghighlight==0] = 0
    gblobs =blob_doh(img, min_sigma = 6, max_sigma = 9, threshold = 0.02, overlap=0.5)


    ngblobs = len(gblobs)
    npblobs = len(pblobs) 
    num_blobs = ngblobs + npblobs
            
    plas_on_glas_dists =[connect((pblob[1], pblob[0]),m_bond, gblobs) for pblob in pblobs]
    num_mconnections = reducer(plas_on_glas_dists)


    plas_on_plas_dists =[connect((pblobs[i][1], pblobs[i][0]),p_bond, pblobs[i+1:]) for i in range(npblobs-1)]
    num_pconnections = reducer(plas_on_plas_dists)



    num_connections = sum([num_pconnections,num_mconnections])
    
    #----------------------filter------------------------- can use actual filter
    test = (num_connections >= expected_connections and num_blobs>=expected_blobs)
    if test: # keep frames that have enough paricles, and are not transitions
        clas=sideify(num_pconnections, num_mconnections )
        
        classes[clas]+=1
            #print(clas)
        
        if clas == 'ucf':
            #broken_count += 1
            #class_num = 0
            #print('ucf')
            pass
        else:
            if origin == 'ucf':
                origin = last_class
                #print("reset origin", origin)
            if clas == last_class:
                class_num += 1
                #print("class_num: %s" %class_num)
            else:
                class_num = 0
                #print("class_reset")
                #print("class_num: %s" %class_num)
            if class_num >= class_thresh: 
                if broken_count >= transition_threshold or clas != origin:
                    transitions[origin + "->" + clas]+=1
                    
                    between_time = frame_no - state_start                               
                    times[origin + "->" + clas][0].append(between_time )

                    stable_time = last_whole - state_start
                    times[origin + "->" + clas][1].append(stable_time)

                    unstable_time = frame_no - last_whole                      
                    times[origin + "->" + clas][2].append(unstable_time)
                
                    state_start = frame_no
                    
                    #print(origin + "->" + clas)
                    origin = clas
                    #print("origin: %s" %origin)
                broken_count = 0
                #print("broken_count: %s" %broken_count)    
                last_whole = frame_no
                
                
            filtrate_len += 1 
            last_class = clas
            
            
    else:
        #print("broken")
        class_num = 0
        #print("class_num: %s" %class_num)
        broken_count += 1
        #print("broken_count: %s" %broken_count)

    return [broken_count, class_num, origin, last_class, last_whole,state_start, filtrate_len]



        
# take largest isodata image region, and return aclassification
def sideify( num_pconnections, num_mconnections):    
    if num_mconnections == 2 and num_pconnections == 3:# and num_mconnections==4: #gtotallengths>48
        return "t"
    elif num_mconnections == 3 and num_pconnections == 2:# and num_mconnections == 4:
        return "c"
    else:
        return "ucf"

def reducer(connections):
    return sum(connections)    
def connect(cv, radius, bloblist): # center of blob2, radius of all blobs, list of other blobs
    num_connections = 0
    norm_radius = 15
    for blob2 in bloblist:
        cv2 = (blob2[1], blob2[0]) # center of blob2
        vector_d = vector_dist(cv, cv2)
        if vector_d <= 2*radius: # blob centers closer than diameter of one blob
            num_connections += 1
        else:
            pass
    return (num_connections)

def vector_dist(v1, v2): # euclidean distance between 2 points
    return math.sqrt(np.sum([(v1[i] - v2[i])**2 for i in range(len(v1))]))

def third_item(l1):
    return l1[2]   

filenames = glob("./tchakamau/W*plastic/4*_14.avi")# +glob("./tchakamau/W*half/*_[1][0,1,2,3,4,7,8,9]*.avi")  #[1,2,3,4,6,7,8,9]["./tchakamau/New_setup/45681_10Vpp_25.avi"
filenames.sort()
print(filenames)
num_top_keys = 3
num_full_keys = 3
num_tran_keys = 2
trans_threshes = np.linspace(20, 60, 4)#[30, 40, 50]#[5, 8, 10, 19, 27, 38, 52, 60]
class_threshes =[15,22,30]
trans_runs = {}
#trans_fil = 10

ave = np.average
expected_blobs = 4
expected_connections = 5
p_bond = 15*1.4
m_bond = 15*1.65#1.6
    
"""area_threshold = 1800   # must have enough particles
convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
major_thresholds = (59, 76)"""
thresholds = [expected_blobs , expected_connections, p_bond, m_bond]
params = [3, 115, 98, 2, 95, 80]
print("Number files: ", len(filenames))
index = 0
for trans_fil in trans_threshes:
    
    for class_thresh in class_threshes:
        
        Top_bottoms = np.zeros((len(filenames), num_top_keys))
        Full_classif = np.zeros((len(filenames), num_full_keys))
        N_transitions = np.zeros((len(filenames), num_tran_keys, num_tran_keys))
        num_total_frames = 0 
        filtrate_len = 0
        for vidnum in range(len(filenames)):
            print("Processing vid %s : %s" %(vidnum, filenames[vidnum]))
            #meta = ffprobe(filenames[vidnum])
            #print(meta)#['@nb_frames'])
            #print(5)
            frames = vreader(filenames[vidnum])
            #frame_vid = frames[:, :, :, 2]# making videos ino a frame list
            #print(frames)



            sides = {"t":[], "c":[], "ucf":[]}
            classes = {"t":[], "c":[], "ucf":[]}
            transitions = [("t->t",[]), ("t->c",[]),
                          ("c->t" ,[]), ("c->c",[])]

            transitions = OrderedDict(transitions)
            broken_count = 0
            class_num = 0
            origin = 'ucf'
            last_class = 'ucf'
            last_whole = 0
            for fr in frames:
                frame = fr[:, :, 2]
                num_total_frames += 1
                org = total_threshold_filter(frame, num_total_frames, trans_fil, class_thresh,
                                             broken_count, class_num, origin, last_class, last_whole, filtrate_len, 
                                             sides, classes, transitions)
                broken_count, class_num, origin, last_class, last_whole, filtrate_len = org
            filtered_len = filtrate_len
            #----------------------------------------------------------------
            trans_len = len([y for x in transitions.values() for y in x])
            skeys = list(sides.keys())
            ckeys = list(classes.keys())
            tkeys = list(transitions.keys())
            slcs = [len(sides[x]) for x in skeys]
            clcs = [len(classes[x]) for x in ckeys]
            trans = [len(transitions[x]) for x in tkeys] #num of each transition
            ucf = len(sides["ucf"])


            Top_bottoms[vidnum,:] = [slcs[i] for i in range(len(skeys))]
            Full_classif[vidnum,:] = [clcs[i] for i in range(len(ckeys))]
            N_transitions[vidnum,:] = np.reshape(trans, (num_tran_keys, num_tran_keys))
            print("T threshold: %s" %trans_fil)
            print("C threshold: %s" %class_thresh)
            print(N_transitions[vidnum,:])
            #print("finished vid %s" %vidnum)
        trans_runs[index] = [[Top_bottoms, Full_classif, N_transitions, (trans_fil, class_thresh), num_top_keys, num_full_keys,
                                 num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames], thresholds, params]
        index += 1
        #---------------------------------------------------------------------------------

#-----------------------------------------------------------------------

np.save('trans_runs_plas15.5.npy',trans_runs)


In [4]:
"""Pipeline for lists of frames"""
""""""
"""Pipeline for videos:"""
# initialize frame lists
filenames = glob("./tchakamau/De*ethylene/*avi")# +glob("./tchakamau/W*half/*_[1][0,1,2,3,4,7,8,9]*.avi")  #[1,2,3,4,6,7,8,9]["./tchakamau/New_setup/45681_10Vpp_25.avi"
filenames.sort()
print(filenames)
num_top_keys = 3
num_full_keys = 3
num_tran_keys = 2
num_lengths = 3
trans_threshes = np.linspace(30, 60, 2)#[30, 40, 50]#[5, 8, 10, 19, 27, 38, 52, 60]
class_threshes =[20,30]
trans_runs = {}
#trans_fil = 10

ave = np.average
expected_blobs = 4
expected_connections = 5
p_bond = 15*1
m_bond = 15*1.1
    
"""area_threshold = 1800   # must have enough particles
convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
major_thresholds = (59, 76)"""
thresholds = [expected_blobs , expected_connections, p_bond, m_bond]
params = [3, 115, 98, 2, 95, 80]
print("Number files: ", len(filenames))
index = 0
for trans_fil in trans_threshes:
    
    for class_thresh in class_threshes:
        
        Full_classif = np.zeros((len(filenames), num_full_keys))
        N_transitions = np.zeros((len(filenames), num_tran_keys, num_tran_keys))
        T_transitions = [[[[[] for i in range(num_tran_keys)] for j in range(num_tran_keys)] for k in range(num_lengths)] 
                          for l in range(len(filenames))]
        num_total_frames = 0 
        filtrate_len = 0
        for vidnum in range(len(filenames)):
            print("Processing vid %s : %s" %(vidnum, filenames[vidnum]))
            #meta = ffprobe(filenames[vidnum])
            #print(meta)#['@nb_frames'])
            #print(5)
            frames = vreader(filenames[vidnum])
            #frame_vid = frames[:, :, :, 2]# making videos ino a frame list
            #print(frames)



            
            classes = [("t",0), ("c",0), ("ucf",0)]
            transitions = [("t->t",0), ("t->c",0),
                          ("c->t" ,0), ("c->c",0)]
            times = [("t->t",[[] for i in range(num_lengths)]), ("t->c",[[] for i in range(num_lengths)]),
                          ("c->t" ,[[] for i in range(num_lengths)]), ("c->c",[[] for i in range(num_lengths)])]

            transitions = OrderedDict(transitions)
            times = OrderedDict(times)            
            classes = OrderedDict(classes)
            broken_count = 0
            class_num = 0
            origin = 'ucf'
            last_class = 'ucf'
            last_whole = 0
            state_start = 0
            frame_no = 0
            for fr in frames:
                frame = fr[:, :, 2]
                frame_no += 1
                num_total_frames += 1
                org = total_threshold_filter(frame, frame_no, trans_fil, class_thresh,
                                             broken_count, class_num, origin, last_class, last_whole, state_start, filtrate_len, 
                                              classes, transitions, times)
                broken_count, class_num, origin, last_class, last_whole, state_start, filtrate_len = org
            filtered_len = filtrate_len
            #----------------------------------------------------------------
             
            ckeys = list(classes.keys())
            clcs = [classes[x] for x in ckeys] # list of frame nums per class
            
            tkeys = list(times.keys())
            tims = [times[x] for x in tkeys]
            
            trkeys = list(transitions.keys())
            trans = [transitions[x] for x in trkeys] #num of each transition
            #ucf = len(sides["ucf"])

            #Top_bottoms[vidnum,:] = [slcs[i] for i in range(len(skeys))]
            Full_classif[vidnum,:] = clcs
            N_transitions[vidnum,:] = np.reshape(trans, (num_tran_keys, num_tran_keys))             
            T_transitions[vidnum] = np.moveaxis(np.reshape(tims, (num_tran_keys, num_tran_keys,num_lengths)),
                                                  [0,1,2], [1,2,0])
            print("T threshold: %s" %trans_fil)
            print("C threshold: %s" %class_thresh)
            print(N_transitions[vidnum,:])
            print(T_transitions[vidnum])
            print(Full_classif[vidnum,:])
        trans_runs[index] = [[Full_classif, (N_transitions, T_transitions), (trans_fil, class_thresh), num_top_keys, num_full_keys,
                                 num_tran_keys, ckeys, num_tran_keys, num_total_frames], thresholds, params]
        index += 1
        #---------------------------------------------------------------------------------

#-----------------------------------------------------------------------

np.save('trans_runs_deethylene_timesbetween.npy',trans_runs)


['./tchakamau\\Density_matchedethylene\\45693_15Vpp_10.avi', './tchakamau\\Density_matchedethylene\\45693_15Vpp_11_pushedatstart.avi', './tchakamau\\Density_matchedethylene\\45693_15Vpp_12.avi', './tchakamau\\Density_matchedethylene\\45693_15Vpp_5.avi', './tchakamau\\Density_matchedethylene\\45693_15Vpp_6.avi', './tchakamau\\Density_matchedethylene\\45693_15Vpp_7.avi', './tchakamau\\Density_matchedethylene\\45693_15Vpp_8_pushedatstart.avi', './tchakamau\\Density_matchedethylene\\45693_15Vpp_9.avi', './tchakamau\\Density_matchedethylene\\45696_15Vpp_0.avi', './tchakamau\\Density_matchedethylene\\45696_15Vpp_1.avi', './tchakamau\\Density_matchedethylene\\45696_15Vpp_2.avi', './tchakamau\\Density_matchedethylene\\45696_15Vpp_3.avi', './tchakamau\\Density_matchedethylene\\45722_15Vpp_4.avi']
Number files:  13
Processing vid 0 : ./tchakamau\Density_matchedethylene\45693_15Vpp_10.avi
T threshold: 30.0
C threshold: 20
[[25.  5.]
 [ 5.  6.]]
[[[list([1908, 841, 1189, 1154, 928, 1163, 430, 430,

T threshold: 30.0
C threshold: 30
[[17.  6.]
 [ 7.  1.]]
[[[list([2583, 2443, 590, 455, 502, 407, 545, 1457, 6395, 592, 162, 2268, 343, 1161, 692, 145, 4679])
   list([1912, 626, 1604, 185, 752, 419])]
  [list([678, 3449, 683, 2838, 42, 821, 916]) list([1131])]]

 [[list([702, 1851, 188, 233, 433, 18, 300, 1343, 6270, 17, 46, 1791, 81, 807, 310, 2, 3940])
   list([696, 383, 729, 21, 618, 251])]
  [list([120, 2775, 129, 2172, 8, 161, 11]) list([671])]]

 [[list([1881, 592, 402, 222, 69, 389, 245, 114, 125, 575, 116, 477, 262, 354, 382, 143, 739])
   list([1216, 243, 875, 164, 134, 168])]
  [list([558, 674, 554, 666, 34, 660, 905]) list([460])]]]
[25165.  6594.  1807.]
Processing vid 2 : ./tchakamau\Density_matchedethylene\45693_15Vpp_12.avi
T threshold: 30.0
C threshold: 30
[[11.  9.]
 [10. 18.]]
[[[list([281, 1864, 1273, 184, 538, 352, 157, 1397, 731, 401, 3215])
   list([308, 6699, 571, 1049, 1448, 117, 3813, 1463, 1085])]
  [list([1276, 882, 811, 288, 669, 197, 556, 3244, 1019, 2055]

T threshold: 60.0
C threshold: 20
[[12.  2.]
 [ 2.  6.]]
[[[list([7658, 767, 859, 1247, 275, 261, 1213, 1260, 1081, 565, 169, 1122])
   list([1223, 928])]
  [list([1565, 808]) list([1389, 982, 3241, 655, 238, 512])]]

 [[list([7407, 72, 420, 128, 139, 16, 0, 1054, 44, 410, 10, 20])
   list([56, 60])]
  [list([775, 132]) list([1257, 334, 2683, 545, 46, 13])]]

 [[list([251, 695, 439, 1119, 136, 245, 1213, 206, 1037, 155, 159, 1102])
   list([1167, 868])]
  [list([790, 676]) list([132, 648, 558, 110, 192, 499])]]]
[10833.  5901.   492.]
Processing vid 4 : ./tchakamau\Density_matchedethylene\45693_15Vpp_6.avi
T threshold: 60.0
C threshold: 20
[[3. 0.]
 [0. 0.]]
[[[list([4964, 3503, 670]) list([])]
  [list([]) list([])]]

 [[list([3964, 420, 366]) list([])]
  [list([]) list([])]]

 [[list([1000, 3083, 304]) list([])]
  [list([]) list([])]]]
[4.776e+03 2.000e+00 1.780e+02]
Processing vid 5 : ./tchakamau\Density_matchedethylene\45693_15Vpp_7.avi
T threshold: 60.0
C threshold: 20
[[4. 1.]
 [0

T threshold: 60.0
C threshold: 30
[[6. 0.]
 [1. 1.]]
[[[list([1741, 434, 3232, 1613, 3898, 1140]) list([])]
  [list([2058]) list([6261])]]

 [[list([1045, 84, 2, 497, 722, 190]) list([])]
  [list([352]) list([2089])]]

 [[list([696, 350, 3230, 1116, 3176, 950]) list([])]
  [list([1706]) list([4172])]]]
[3307. 2533.  137.]
Processing vid 8 : ./tchakamau\Density_matchedethylene\45696_15Vpp_0.avi
T threshold: 60.0
C threshold: 30
[[0. 1.]
 [1. 0.]]
[[[list([]) list([559])]
  [list([1605]) list([])]]

 [[list([]) list([367])]
  [list([1017]) list([])]]

 [[list([]) list([192])]
  [list([588]) list([])]]]
[1878. 1031.  142.]
Processing vid 9 : ./tchakamau\Density_matchedethylene\45696_15Vpp_1.avi
T threshold: 60.0
C threshold: 30
[[5. 0.]
 [0. 0.]]
[[[list([2093, 1394, 999, 5933, 686]) list([])]
  [list([]) list([])]]

 [[list([1547, 134, 435, 5496, 53]) list([])]
  [list([]) list([])]]

 [[list([546, 1260, 564, 437, 633]) list([])]
  [list([]) list([])]]]
[7.769e+03 1.000e+00 9.640e+02]
Pr